# Intro a los modelos de clasificación

Se construirán modelos de clasificación a partir de un conjunto de algoritmos de inferencia y el dataset de Iris, los cuales serán evaluados con los siguientes métodos:
- 1 clasificador y hold-out
- Múltiples clasificadores y hold-out
- 1 clasificador y validación cruzada
- Múltiples clasificadores y validación cruzada

Recordar que los modelos una vez evaluados, pueden ser reentrenados con el conjunto completo de datos para hacer predicciones posteriores.

### Datos

In [1]:
# Importar datos
from sklearn.datasets import load_iris
iris = load_iris()

# Crear un DataFrame de Pandas
import pandas as pd
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['species'] = iris.target

# Separar vector de características (X_i) y etiquetas (y)
X = df.drop('species', axis=1)
y = df['species']

### Algoritmos de inferencia

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Establecer algoritmos de inferencia
models = {
    "Naive Bayes": GaussianNB(),
    "C4.5 (Decision Tree)": DecisionTreeClassifier(),
    "k-Nearest Neighbors": KNeighborsClassifier(),
    "Support Vector Machine": SVC()
}

## Hold-out validation

In [3]:
# Partición de los datos 80% training, 20% testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(120, 4) (120,)
(30, 4) (30,)


### Un modelo

In [4]:
# Construir modelo con datos de training
clf = GaussianNB().fit(X_train, y_train)

# Evaluar modelo con datos de testing (mostrar accuracy)
clf.score(X_test, y_test)

0.9666666666666667

### Varios modelos

In [5]:
results = {} 
for name, model in models.items():
    clf = model.fit(X_train, y_train)
    cv_results = clf.score(X_test, y_test)
    results[name] = cv_results
    print(f"{name}: {cv_results.mean():.4f}")

Naive Bayes: 0.9667
C4.5 (Decision Tree): 1.0000
k-Nearest Neighbors: 0.9667
Support Vector Machine: 1.0000


## Validación cruzada

### Un modelo

*** Forma simple: cross_val_score

In [6]:
from sklearn.model_selection import cross_val_score

clf = GaussianNB()

# Training y testing en una sola función
scores = cross_val_score(clf, X, y, cv=5)

print("Resultados para cada iteración: ", scores)

print("Resultado promedio: ", scores.mean())

Resultados para cada iteración:  [0.93333333 0.96666667 0.93333333 0.93333333 1.        ]
Resultado promedio:  0.9533333333333334


*** Otra forma: kfold (con .fit  y .score)

In [7]:
from sklearn.model_selection import StratifiedKFold

# Realizar particiones de los datos 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Visualizar los índices para las particiones
#for train, test in skf.split(X, y): print("%s %s" % (train, test))

In [8]:
acc_scores = []

for train_index, test_index in skf.split(X, y):
    # Particiones para training
    X_train_fold = X.iloc[train_index]
    y_train_fold = y.iloc[train_index]
    # Particiones para testing
    X_test_fold = X.iloc[test_index]
    y_test_fold = y.iloc[test_index]
    # Entrenamiento del modelo
    clf = GaussianNB().fit(X_train_fold, y_train_fold)
    # Evaluación de modelo y se acumula el accuracy
    acc = clf.score(X_test_fold, y_test_fold)
    print("Fold accuracy: ", acc)
    acc_scores.append(acc)    

from statistics import mean, stdev
print('Overall Accuracy:', mean(acc_scores))
print('Standard Deviation is:', stdev(acc_scores))

Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9333333333333333
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9666666666666667
Overall Accuracy: 0.96
Standard Deviation is: 0.014907119849998594


### Varios modelos

In [9]:
from sklearn.metrics import accuracy_score

# Iterar sobre los modelos
for name, model in models.items():
    acc_scores = []
    # Iterar sobre las particiones de los datos
    for train_index, test_index in skf.split(X, y):
        # Particiones para training y testing
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        # Entrenamiento y evaluación del modelo
        model.fit(X_train_fold, y_train_fold)
        y_predict = model.predict(X_test_fold)
        acc = accuracy_score(y_test_fold, y_predict)
        print("Fold accuracy: ", acc)
        acc_scores.append(acc)
        
    avg_acc = sum(acc_scores)/len(acc_scores)
    print(f"{name}: Avg accuracy {avg_acc:.4f}")

Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9333333333333333
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9666666666666667
Naive Bayes: Avg accuracy 0.9600
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.8666666666666667
Fold accuracy:  1.0
Fold accuracy:  0.9333333333333333
C4.5 (Decision Tree): Avg accuracy 0.9467
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9333333333333333
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9333333333333333
k-Nearest Neighbors: Avg accuracy 0.9533
Fold accuracy:  1.0
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9333333333333333
Fold accuracy:  0.9666666666666667
Fold accuracy:  0.9
Support Vector Machine: Avg accuracy 0.9533
